In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import *
from sklearn import svm
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import *
from sklearn.decomposition import *
from sklearn.impute import *
from sklearn.pipeline import Pipeline
from sklearn.datasets import *
from sklearn.feature_selection import *
import numpy as np
import random
import time
import csv

##from decisionTree import saudacao

In [4]:
## Importante sempre especificar a seed para garantir a reproducilidade dos resultados
## No caso do sklearn tem que especificar o random_state
RANDOM_STATE = 42

In [5]:
df = pd.read_csv("arrhythmia.csv", header=None).replace("?", np.nan)
data = df.to_numpy()
x, y = data[:, :-1], data[:, -1]
y = pd.Series(y).apply(str)
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.8,test_size=0.2,stratify=y, random_state=RANDOM_STATE)

#Teste pra corrigir aviso de UserWarning: Features [ 19  21  47  67  69  83  85 131 132 139 141 143 144 145 151 156 157 164 204 264 274] are constant.
selector = VarianceThreshold(threshold=0.01)
x_train = selector.fit_transform(x_train)
x_test = selector.transform(x_test)

In [6]:
def criar_individuo_decisionTree():
    
    start_time = time.time()
    # Parametros do StandardScaler
    with_mean = random.choice([True, False])
    with_std = random.choice([True, False])
    
    # Parametros do SimpleImputer
    imputer_strategies = ["mean", "median", "most_frequent", "constant"]
    imputer_strategy = random.choice(imputer_strategies)
    
    if imputer_strategy == "constant":
        fill_value = random.uniform(0.0, 1.0)
    else:
        fill_value = None
    
    # Parametros do SelectKBest
    k_best_params = {
        "k": random.randint(10, 50)
    }
    
    # Parametros do classificador (por exemplo, DecisionTreeClassifier)
    clf_params = {
        "max_depth": random.randint(1, 10),
        "min_samples_split": random.randint(2, 20), 
        "random_state": RANDOM_STATE
    }
    
    individuo = [
        ('scaler', StandardScaler(with_mean=with_mean, with_std=with_std)),
        ('imputer', SimpleImputer(strategy=imputer_strategy, fill_value=fill_value)),
        ('feature-selection', SelectKBest(**k_best_params)),
        ('classifier', DecisionTreeClassifier(**clf_params))
    ]
    
    individuo_dict = {
        'scaler_params': {'with_mean': with_mean, 'with_std': with_std},
        'imputer_params': {'strategy': imputer_strategy, 'fill_value': fill_value},
        'feature_selection_params': k_best_params,
        'classifier_params': clf_params
    }
    
    return individuo, start_time, individuo_dict

In [7]:
#scaler_params = {"with_mean":with_mean, "with_std":with_std}
#imputer_params = {"strategy": imputer_strategy, "fill_value": fill_value}
#feature_selection_params = {"k": 31}
#clf_params = {"max_depth": 10, "min_samples_split": 2, "random_state": RANDOM_STATE}

column_names = ['Individuo', 'Tempo de execucao', 'Accuracy', 'F1 Score']

for _ in range(10):
    individuo, start_time, individuo_dict = criar_individuo_decisionTree()
    pipeline = Pipeline(individuo)
    pipeline.fit(x_train, y_train)
    
    #predições
    predictions = pipeline.predict(x_test)
    
    f1 = f1_score(y_test, predictions, average='weighted', zero_division=1.0)

    accuracy = pipeline.score(x_test, y_test)
    report = classification_report(y_test, predictions)
    
    end_time = time.time()
    tempo = end_time - start_time

    with open('resultados_decisionTree.csv', mode='a', newline='') as file:
        writer = csv.writer(file)
        
        if file.tell() == 0:
            writer.writerow(column_names)
        writer.writerow([individuo_dict, tempo, accuracy, f1])

C:\Users\pcdaf\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\pcdaf\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\pcdaf\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\pcdaf\anaconda3\Lib\site-packages\sklearn\metr

In [8]:
def criar_individuo_randomForest():
    
    start_time = time.time()
    # Parametros do StandardScaler
    with_mean = random.choice([True, False])
    with_std = random.choice([True, False])
    
    # Parametros do SimpleImputer
    imputer_strategies = ["mean", "median", "most_frequent", "constant"]
    imputer_strategy = random.choice(imputer_strategies)
    
    if imputer_strategy == "constant":
        fill_value = random.uniform(0.0, 1.0)
    else:
        fill_value = None
    
    # Parametros do SelectKBest
    k_best_params = {
        "k": random.randint(10, 50)
    }
    
    clf_params = {
        "n_estimators": random.randint(1, 100),
        "max_depth": random.randint(1, 20), 
        "min_samples_split": random.randint(2, 20), 
        "random_state": RANDOM_STATE
    }
    
    individuo = [
        ('scaler', StandardScaler(with_mean=with_mean, with_std=with_std)),
        ('imputer', SimpleImputer(strategy=imputer_strategy, fill_value=fill_value)),
        ('feature-selection', SelectKBest(**k_best_params)),
        ('classifier', RandomForestClassifier(**clf_params))
    ]
    
    individuo_dict = {
        'scaler_params': {'with_mean': with_mean, 'with_std': with_std},
        'imputer_params': {'strategy': imputer_strategy, 'fill_value': fill_value},
        'feature_selection_params': k_best_params,
        'classifier_params': clf_params
    }
    
    return individuo, start_time, individuo_dict

In [9]:
column_names = ['Individuo', 'Tempo de execucao', 'Accuracy', 'F1 Score']

for _ in range(10):
    individuo, start_time, individuo_dict = criar_individuo_randomForest()
    pipeline = Pipeline(individuo)
    pipeline.fit(x_train, y_train)
    
    #predições
    predictions = pipeline.predict(x_test)
    
    f1 = f1_score(y_test, predictions, average='weighted', zero_division=1.0)

    accuracy = pipeline.score(x_test, y_test)
    report = classification_report(y_test, predictions)
    
    end_time = time.time()
    tempo = end_time - start_time

    with open('resultados_randomForest.csv', mode='a', newline='') as file:
        writer = csv.writer(file)
        
        if file.tell() == 0:
            writer.writerow(column_names)
        writer.writerow([individuo_dict, tempo, accuracy, f1])

##scaler_params = {"with_mean":True, "with_std":True}
##imputer_params = {"strategy": "most_frequent"}
##feature_selection_params = {"k": 36}
##clf_params = {"n_estimators": 10, "max_depth": 13, "min_samples_split": 2, "random_state": RANDOM_STATE}

C:\Users\pcdaf\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\pcdaf\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\pcdaf\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\pcdaf\anaconda3\Lib\site-packages\sklearn\metr

In [10]:
def criar_individuo_SVC():
    
    start_time = time.time()
    # Parametros do StandardScaler
    with_mean = random.choice([True, False])
    with_std = random.choice([True, False])
    
    # Parametros do SimpleImputer
    imputer_strategies = ["mean", "median", "most_frequent", "constant"]
    imputer_strategy = random.choice(imputer_strategies)
    
    if imputer_strategy == "constant":
        fill_value = random.uniform(0.0, 1.0)
    else:
        fill_value = None
    
    # Parametros do SelectKBest
    k_best_params = {
        "k": random.randint(10, 50)
    }
    
    clf_params = {
        "C": random.uniform(0.1, 2.0),
        "kernel": random.choice(["linear", "poly", "rbf", "sigmoid"]),
        "gamma": random.choice(["scale", "auto"]),
        "tol": 10**(-random.randint(3, 7)), 
        "random_state": RANDOM_STATE
    }
    
    individuo = [
        ('scaler', StandardScaler(with_mean=with_mean, with_std=with_std)),
        ('imputer', SimpleImputer(strategy=imputer_strategy, fill_value=fill_value)),
        ('feature-selection', SelectKBest(**k_best_params)),
        ('classifier', svm.SVC(**clf_params))
    ]
    
    individuo_dict = {
        'scaler_params': {'with_mean': with_mean, 'with_std': with_std},
        'imputer_params': {'strategy': imputer_strategy, 'fill_value': fill_value},
        'feature_selection_params': k_best_params,
        'classifier_params': clf_params
    }
    
    return individuo, start_time, individuo_dict

In [11]:
##scaler_params = {"with_mean":True, "with_std":True}
##imputer_params = {"strategy": "median"}
##feature_selection_params = {"k": 37}
##clf_params = {"C": 1, "kernel": "sigmoid", "gamma": "auto", "tol": 10E-5, "random_state": RANDOM_STATE}

for _ in range(10):
    individuo, start_time, individuo_dict = criar_individuo_SVC()
    pipeline = Pipeline(individuo)
    pipeline.fit(x_train, y_train)
    
    #predições
    predictions = pipeline.predict(x_test)
    
    f1 = f1_score(y_test, predictions, average='weighted', zero_division=1.0)

    accuracy = pipeline.score(x_test, y_test)
    report = classification_report(y_test, predictions)
    
    end_time = time.time()
    tempo = end_time - start_time

    with open('resultados_SVC.csv', mode='a', newline='') as file:
        writer = csv.writer(file)
        
        if file.tell() == 0:
            writer.writerow(column_names)
        writer.writerow([individuo_dict, tempo, accuracy, f1])

C:\Users\pcdaf\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\pcdaf\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\pcdaf\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\pcdaf\anaconda3\Lib\site-packages\sklearn\metrics\_cla

In [12]:
def criar_individuo_AdaBoost():
    
    start_time = time.time()
    # Parametros do StandardScaler
    with_mean = random.choice([True, False])
    with_std = random.choice([True, False])
    
    # Parametros do SimpleImputer
    imputer_strategies = ["mean", "median", "most_frequent", "constant"]
    imputer_strategy = random.choice(imputer_strategies)
    
    if imputer_strategy == "constant":
        fill_value = random.uniform(0.0, 1.0)
    else:
        fill_value = None
    
    # Parametros do SelectKBest
    k_best_params = {
        "k": random.randint(10, 50)
    }
    
    clf_params = {
        "n_estimators": random.randint(1, 100),
        "random_state": RANDOM_STATE
    }
    
    individuo = [
        ('scaler', StandardScaler(with_mean=with_mean, with_std=with_std)),
        ('imputer', SimpleImputer(strategy=imputer_strategy, fill_value=fill_value)),
        ('feature-selection', SelectKBest(**k_best_params)),
        ('classifier', AdaBoostClassifier(**clf_params))
    ]
    
    individuo_dict = {
        'scaler_params': {'with_mean': with_mean, 'with_std': with_std},
        'imputer_params': {'strategy': imputer_strategy, 'fill_value': fill_value},
        'feature_selection_params': k_best_params,
        'classifier_params': clf_params
    }
    
    return individuo, start_time, individuo_dict

In [13]:
for _ in range(100):
    individuo, start_time, individuo_dict = criar_individuo_AdaBoost()
    pipeline = Pipeline(individuo)
    pipeline.fit(x_train, y_train)
    
    #predições
    predictions = pipeline.predict(x_test)
    
    f1 = f1_score(y_test, predictions, average='weighted', zero_division=1.0)

    accuracy = pipeline.score(x_test, y_test)
    report = classification_report(y_test, predictions)
    
    end_time = time.time()
    tempo = end_time - start_time

    with open('resultados_AdaBoost.csv', mode='a', newline='') as file:
        writer = csv.writer(file)
        
        if file.tell() == 0:
            writer.writerow(column_names)
        writer.writerow([individuo_dict, tempo, accuracy, f1])

C:\Users\pcdaf\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\pcdaf\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\pcdaf\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\pcdaf\anaconda3\Lib\site-packages\sklearn\metr

In [17]:
pip install deap


     ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
     - -------------------------------------- 0.0/1.1 MB ? eta -:--:--
     ---- ----------------------------------- 0.1/1.1 MB 1.3 MB/s eta 0:00:01
     -------- ------------------------------- 0.2/1.1 MB 1.8 MB/s eta 0:00:01
     ----------- ---------------------------- 0.3/1.1 MB 2.0 MB/s eta 0:00:01
     ------------------------- -------------- 0.7/1.1 MB 3.0 MB/s eta 0:00:01
     ---------------------------- ----------- 0.8/1.1 MB 2.8 MB/s eta 0:00:01
     ---------------------------------------  1.0/1.1 MB 3.3 MB/s eta 0:00:01
     ---------------------------------------- 1.1/1.1 MB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for deap: filename=deap-1.4.1-py3-none-any.whl size=97349 sha256=2ebcf0d278d42e3e4fa09123a58c257750a4916f7a6d10ae36ddee785a99bc7c
  Stored in directory: c:\users\pcdaf\appdata\local\pip\cache\wh

In [18]:
from deap import base, creator, tools, algorithms

In [31]:
# Definir o tipo de problema (maximização)
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

# Inicialização das configurações do DEAP
toolbox = base.Toolbox()

#Definir hiperparâmetros a serem otimizados
toolbox.register("n_estimators", random.randint, 1, 100)
toolbox.register("max_depth", random.randint, 1, 20)
toolbox.register("min_samples_split", random.uniform, 0.01, 1.0)
toolbox.register("min_samples_leaf", random.uniform, 0.01, 0.5)

def create_individual():
    return [toolbox.n_estimators(), toolbox.max_depth(), toolbox.min_samples_split(), toolbox.min_samples_leaf()]

scaler_params = {"with_mean": True, "with_std": True}
imputer_params = {"strategy": 'mean', "fill_value": None}
feature_selection_params = {"k": 10}
clf_params = {'n_estimators': 18, "max_depth": 17, "min_samples_split": 4, "random_state": RANDOM_STATE}

toolbox.register("individual", tools.initIterate, creator.Individual, create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def evaluate(individual):
    n_estimators, max_depth, min_samples_split, min_samples_leaf = individual
    clf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, min_samples_split=min_samples_split)
    iris = load_iris()
    scores = cross_val_score(clf, data, cv=5)
    accuracy = scores.mean()
    return accuracy

toolbox.register("evaluate", evaluate)

# Definir operadores genéticos
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt, low=[1, 1, 0.01, 0.01], up=[100, 20, 1.0, 0.5], indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

# Criar uma população inicial
population = toolbox.population(n=50)

# Rodar o algoritmo genético
pop, logbook = algorithms.eaMuPlusLambda(population, toolbox, mu=10, lambda_=50, cxpb=0.7, mutpb=0.2, ngen=10, stats=None, verbose=True)

# Obter o melhor indivíduo após a execução do algoritmo
best_individual = tools.selBest(pop, k=1)[0]

print("Melhor Indivíduo:", best_individual)
print("Melhor Valor de Aptidão (Precisão):", best_individual.fitness.values[0])

C:\Users\pcdaf\anaconda3\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
C:\Users\pcdaf\anaconda3\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\pcdaf\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 730, in _fit_and_score
    estimator.fit(X_train, **fit_params)
  File "C:\Users\pcdaf\anaconda3\Lib\site-packages\sklearn\base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: BaseForest.fit() missing 1 required positional argument: 'y'
